# House Price Prediction


**Data Source**
<br>
[https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data)
      
1) train.csv - This is the data that is imported for analysis and modelling
<br>
2) descripion.txt - This file includes description of all columns in dataset
<br>
<br>
**Goal**
<br>
We'll be trying to predict house price with K-Nearest Neighbor regression model.       

### I. Importing Libraries

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

import joblib
import math
import warnings
warnings.filterwarnings(action='ignore')

### II. Loading the House Price Prediction Data 

In [100]:
# We are using the train.csv data file downloaded from kaggle website and check 
# the shape and header of the data read in a dataframe

train_df = pd.read_csv("https://raw.githubusercontent.com/PavithraUmakanthan/DataScienceProject_Python/main/train.csv")


print("Train Data Dimensions :", train_df.shape)
train_df.head()

Train Data Dimensions : (1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [101]:
# Printing the columns names of the data frame
print(train_df.columns.values)

['Id' 'MSSubClass' 'MSZoning' 'LotFrontage' 'LotArea' 'Street' 'Alley'
 'LotShape' 'LandContour' 'Utilities' 'LotConfig' 'LandSlope'
 'Neighborhood' 'Condition1' 'Condition2' 'BldgType' 'HouseStyle'
 'OverallQual' 'OverallCond' 'YearBuilt' 'YearRemodAdd' 'RoofStyle'
 'RoofMatl' 'Exterior1st' 'Exterior2nd' 'MasVnrType' 'MasVnrArea'
 'ExterQual' 'ExterCond' 'Foundation' 'BsmtQual' 'BsmtCond' 'BsmtExposure'
 'BsmtFinType1' 'BsmtFinSF1' 'BsmtFinType2' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' 'Heating' 'HeatingQC' 'CentralAir' 'Electrical' '1stFlrSF'
 '2ndFlrSF' 'LowQualFinSF' 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath'
 'FullBath' 'HalfBath' 'BedroomAbvGr' 'KitchenAbvGr' 'KitchenQual'
 'TotRmsAbvGrd' 'Functional' 'Fireplaces' 'FireplaceQu' 'GarageType'
 'GarageYrBlt' 'GarageFinish' 'GarageCars' 'GarageArea' 'GarageQual'
 'GarageCond' 'PavedDrive' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch'
 '3SsnPorch' 'ScreenPorch' 'PoolArea' 'PoolQC' 'Fence' 'MiscFeature'
 'MiscVal' 'MoSold' 'YrSold' 'SaleTy

**We can make the following observations from above:**

 1) We have 81 columns in our data set
 <br>
 2) Sale Price is our target variable and we need to create a model to predict the same

### III. Data Analysis

In [102]:
# Printing a concise summary of the dataframe including the data types and non null
# count of the columns in the dataframe
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [103]:
# Printing the summary statistics of the num and float datatype columns in the dataframe
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


In [104]:
# Calculating the count of the different columns in the dataframe
data_types = train_df.dtypes
data_types.groupby(data_types).count()

int64      35
float64     3
object     43
dtype: int64

In [105]:
# Extracting numerical variables
numerical_features = train_df.select_dtypes(include = ['int64', 'float64'])

numerical_features.drop(columns=['Id'], inplace=True)
display(numerical_features.head(5))

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000


In [106]:
# Printing the numerical column names in dataframe
print(numerical_features.columns.values)

['MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2'
 'BsmtUnfSF' 'TotalBsmtSF' '1stFlrSF' '2ndFlrSF' 'LowQualFinSF'
 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath' 'FullBath' 'HalfBath'
 'BedroomAbvGr' 'KitchenAbvGr' 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt'
 'GarageCars' 'GarageArea' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch'
 '3SsnPorch' 'ScreenPorch' 'PoolArea' 'MiscVal' 'MoSold' 'YrSold'
 'SalePrice']


In [ ]:
# Plotting histogram of the numerical variables
fig = plt.figure()
for i, feature in enumerate(numerical_features.columns):
    ax = fig.add_subplot(23,2,i+1)
    train_df[feature].hist(bins=40, ax=ax, color='blue',alpha=0.5, figsize=(40,200))
    ax.set_title(feature, fontsize = 25)
    ax.tick_params(axis = 'both', which = 'major', labelsize = 20)
    ax.tick_params(axis = 'both', which = 'minor', labelsize = 20)
    ax.set_xlabel('')
plt.show()

**From the above plots we can observe that we have two types of numerical variables:**
1) We have continuous numerical variables such as LotFrontage, YearBuilt and also other variables representing SquareFoot
<br>
2) We also have discrete variables such as FullBath, HalfBath, OverallCond etc

In [ ]:
# creating a correlation heatmap to understand the linear relationship between different 
# variables and sale price in the housing data set
corr = train_df.corr()
high_corr_features = corr.index[corr["SalePrice"]>0.5]
plt.subplots(figsize=(15,14))
sns.heatmap(train_df[high_corr_features].corr(),annot=True, cmap="Blues", square=True)

In [ ]:
# Below are the Top 10 correlations
corr = (train_df.corr()['SalePrice']).sort_values(ascending=False)
print(corr.head(11))

From the above correlation we can observe that variables like OverallQual, GrLivArea, GarageArea are strongly correlated 
with sale price

In [ ]:
# # Analyzing target variable
# train_df['SalePrice'].describe()

In [ ]:
# sns.histplot(train_df['SalePrice'])
# print("Skewness: %f" % train_df['SalePrice'].skew())

We can observe above that our target variable, Sale Price, is right skewed. Normal distribution (bell-shaped) of variables is one of the assumptions of regression problems. Skewness between -0.5 and 0.5 is considered almost skewed.

In [ ]:
# # Similarly we will also view skewness in other features or columns
# skewed_columns = pd.DataFrame(train_df.select_dtypes(include=['int64','float64']).skew(), columns=['Skewness'])
# skewed_columns = skewed_columns.sort_values(ascending=False, by='Skewness')
# skewed_columns = skewed_columns[(skewed_columns['Skewness'] > 0.5) | (skewed_columns['Skewness'] < -0.5)]
# skewed_columns

### IV. Data Preprocessing

#### Handling Missing Values

In [ ]:
# We are calculating the number and percentage of missing values in all columns

def missing_values_sum(data):
    missing_data_df = pd.DataFrame({'FeatureName':[], 'TotalMissingValues':[], 'PercentMissingValues':[]})
    for feature in data.columns:
        sum_missing = train_df[feature].isnull().sum()
        missing_data_df.loc[len(missing_data_df.index)] = [feature, sum_missing, round((sum_missing/train_df.shape[0])*100,2)]
    return missing_data_df
        
missing_data_df = missing_values_sum(train_df)
missing_data_df[missing_data_df['TotalMissingValues'] > 0].sort_values(by='TotalMissingValues', ascending=False).style.background_gradient()

As observed above, PoolQC, MiscFeature, Alley and Fence have more than 75% missing values.
Below we will remove those from the dataset.

In [ ]:
# Removing columns with more than 75% missing values
drop_columns = missing_data_df[missing_data_df['PercentMissingValues'] > 75.0].FeatureName.tolist()
print("Columns to drop : ", drop_columns)
train_df = train_df.drop(drop_columns,axis=1)

**We will also drop MiscVal which depends on MiscFeature as evident in description file.**

MiscFeature: Miscellaneous feature not covered in other categories
<br>
MiscVal: $Value of miscellaneous feature

In [ ]:
# Dropping features that are related to the ones we just removed
train_df = train_df.drop(['MiscVal'], axis=1)

FireplacesOu column will have a value 'NA' if the number of fireplaces is zero.
As checked below, the FireplaceQu is null (instead of NA) in rows where fireplaces value is zero

Fireplaces: Number of fireplaces

FireplaceQu: Fireplace quality

       Ex	Excellent - Exceptional Masonry Fireplace
       Gd	Good - Masonry Fireplace in main level
       TA	Average - Prefabricated Fireplace in main living area or Masonry Fireplace in basement
       Fa	Fair - Prefabricated Fireplace in basement
       Po	Poor - Ben Franklin Stove
       NA	No Fireplace

In [ ]:
# Checking if Fireplaces with zero and FireplaceQu with null value have same id
zero_fireplace = train_df[train_df['Fireplaces'] == 0]
null_fireplace_qu = train_df[train_df['FireplaceQu'].isnull()]

#print(zero_fireplace.shape[0], null_fireplace_qu.shape[0])
zero_fireplace['Id'].equals(null_fireplace_qu['Id']) 

In [ ]:
# Replacing the null value in FireplaceQu with 'NA'
train_df.FireplaceQu = train_df.FireplaceQu.fillna('NA')

In [ ]:
# Now from the remaining columns with missing values, we are the calculating the correlation of
# the columns with sale price
missing_data_df = missing_values_sum(train_df)
corr_column_names = missing_data_df[missing_data_df['TotalMissingValues'] > 0].FeatureName.tolist()
corr_column_names.append('SalePrice')
train_df[corr_column_names].corr()

In [ ]:
# As seen above, we can see the below columns are not very correlated with the Sale price and hence we drop 
# those columns

columns_to_remove = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']
train_df = train_df.drop(columns_to_remove,axis=1)

In [ ]:
# As per the data description, the value for the below categorical columns should be
# NA or None if the feature is not available. Example if there is no Basement or Garage
# then the value for Cond or Qual columns should be NA

missing_data_df = missing_values_sum(train_df)
missing_data_columns = missing_data_df[missing_data_df['TotalMissingValues'] > 0].FeatureName.tolist()
for column in missing_data_columns:
    print(column, " : ",train_df[column].unique())

In [ ]:
# We are replacing the null value with NA or None for the above categorical columns as
# given in the data description file

for column in ['GarageType','GarageFinish','BsmtFinType2','BsmtFinType1','BsmtExposure','GarageCond','GarageQual','BsmtCond','BsmtQual']:
    train_df[column].fillna('NA', inplace=True)

train_df['MasVnrType'].fillna('None',inplace=True)

In [ ]:
# Finally we only have one column with missing value, which is Electrical. Since there is only a single 
# row with the missing value we will remove that row from our data

missing_data_df = missing_values_sum(train_df)
print(missing_data_df[missing_data_df['TotalMissingValues'] > 0])
train_df = train_df.dropna(how='any', subset=['Electrical'])

#### Standardize

In [ ]:
# We are seprating the target variable from other features in the dataset
train_target = pd.DataFrame((train_df['SalePrice']),columns=['SalePrice'])
train_df = train_df.drop(columns=['SalePrice'])

In [ ]:
# First we are standardizing the sale price as below
scaler = preprocessing.StandardScaler().fit(train_target)
train_target = pd.DataFrame(scaler.transform(train_target), columns=['SalePrice'])
sns.histplot(train_target)

In [ ]:
# Next we are going to sandardize the remaining continous columns in the dataset
columns_to_scale = ['LotArea','YearBuilt','YearRemodAdd','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF',
                    '2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch',
                    '3SsnPorch','ScreenPorch','YrSold']
train_scale = train_df[columns_to_scale]

In [ ]:
scaler = preprocessing.StandardScaler().fit(train_scale)
train_sc = pd.DataFrame(scaler.transform(train_scale), columns=columns_to_scale)

In [ ]:
train_df.update(train_sc)

#### Encoding Cayegorical Variables

In [ ]:
# Simmilar to OverallQual and OverallCond in the dataset, we will convert few categorical features to an interval scale
# and perform Manual Label Encoding 

train_df = train_df.replace({
    'Utilities': {'ELO':1, 'NoSeWa':2, 'NoSewr':3, 'AllPub':4},
    'ExterQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'ExterCond': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'BsmtQual': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
    'BsmtCond': {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5},
    'BsmtExposure': {'NA':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4},
    'BsmtFinType1': {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
    'BsmtFinType2': {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6},
    'HeatingQC': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'KitchenQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'FireplaceQu': {'NA':0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'GarageFinish': {'NA':0, 'Unf':1, 'RFn':2, 'Fin':3},
    'GarageQual': {'NA':0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5},
    'GarageCond': {'NA':0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5}
})

# Also convert Month Sold from numerical to categorical values
train_df = train_df.replace({
    'MoSold' : {1 : 'Jan', 2 : 'Feb', 3: 'Mar', 4 : 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'},
})

In [ ]:
# One Hot encoding for categorical features
train_df_numerical = train_df.select_dtypes(include=['int64', 'float64'])
train_df_categorical = train_df.select_dtypes(include=['object'])
train_df_one_hot = pd.get_dummies(train_df_categorical, drop_first=True)

In [ ]:
train_df_encoded = pd.concat([train_df_numerical, train_df_one_hot], axis=1)

In [ ]:
train_df_encoded.drop(columns=['Id'],axis=1, inplace=True)

### V. Split Data to train, test and val

In [ ]:
# We are now splitting the data into three sets of train, test and validation with respectic ratio 
# of 70:15:15
# We will use the validation data for hyperparameter tuning to evaluate the Number of neighbors (n_neighbors)
# and then the test data to evaluate the performance of our model

train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15


X_train, X_test, Y_train, Y_test = train_test_split(train_df_encoded, train_target, test_size=1-train_ratio, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)

print("Training Data Shape : ", X_train.shape)
print("Validation Data Shape : ",X_val.shape)
print("Test Data Shape : ",X_test.shape)

### VI. Creating KNN Model

In [ ]:
# Now we are creating our model and evaluating it on the validation set
# We are using n_neighbors as 5. We will later tune this parameter to find a good fit
knn_model = KNeighborsRegressor(n_neighbors = 5,weights = 'uniform')
knn_model.fit(X_train,Y_train)

In [ ]:
# Calculating the KNN Score
score = knn_model.score(X_val, Y_val)
print(score)

In [ ]:
# Calculating the RMSE (Root Mean Squared Error)
Y_val_pred = knn_model.predict(X_val)
mse = mean_squared_error(Y_val, Y_val_pred)
print("Mean Squared Error:",mse)
rmse = math.sqrt(mse)
print("Root Mean Squared Error:", rmse)

### VII. Hyperparameter Tuning

In [ ]:
# We will now do hyperparameter tuning by using different values of K 
# and finding its Training and Validation Scores
# We will then select the value that gives good scores and gives lower variance
# between train and val scores
def get_scores(max_neighbors):
    train_score = []
    val_score = []
    models = []
    
    for neighbors in range(1, max_neighbors+1):
        model = KNeighborsRegressor(n_neighbors=neighbors, weights = 'uniform')
        model.fit(X_train, Y_train)
        train_score.append(model.score(X_train, Y_train))
        val_score.append(model.score(X_val, Y_val))
        models.append(model)
        
    return train_score, val_score, models

In [ ]:
def plot_score(neighbors=15):
    train_score, val_score, models = get_scores(neighbors)
    neighbors = range(1, neighbors+1)
    plt.plot(neighbors, train_score, label="Training scores")
    plt.plot(neighbors, val_score, label="Validation scores")
    plt.xlabel("Neighbors")
    plt.ylabel("Score")
    plt.legend()
    plt.show()
    return train_score, val_score, models 
train_score, val_score, models = plot_score()

In [ ]:
print("K=10 Val Score : " , val_score[9])
print("K=10 Train Score : " , train_score[9])

As we can see above in plot, K = 10 appears to be a good fit for our model with bias but lower variance 

In [ ]:
# We will first concat training and validaton dataset and then create a KNN model with K = 10 and dataset
# and then save the model with joblib

X_full = pd.concat([X_train, X_val])
Y_full = pd.concat([Y_train, Y_val])

knn_model_fin = KNeighborsRegressor(n_neighbors = 10,weights = 'uniform')
knn_model_fin.fit(X_full,Y_full)

pwd = %pwd
!mkdir knn_model
knn_model_name = f'{pwd}/knn_model/knn_model_final.sav'
joblib.dump(knn_model_fin,knn_model_name)

### VIII. Evaluate Test Score with Model

In [ ]:
# Loading the saved model
kNN_loaded = joblib.load(f'{pwd}/knn_model/knn_model_final.sav')

In [ ]:
score = kNN_loaded.score(X_test, Y_test)
print("KNN Score : ", score)

In [ ]:
Y_test_pred = kNN_loaded.predict(X_test)
mse = mean_squared_error(Y_test, Y_test_pred)
print("Mean Squared Error:",mse)
rmse = math.sqrt(mse)
print("Root Mean Squared Error:", rmse)